In [1]:
val x = 2
val y = 3
x * y

Intitializing Scala interpreter ...

Spark Web UI available at http://MZC01-MINJIWOO:4040
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1695098015889)
SparkSession available as 'spark'


x: Int = 2
y: Int = 3
res0: Int = 6


In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

// spark session

// load data 
val df = spark.read
    .format("csv")
    .option("format", "csv")
    .option("sep", "\t")
    .option("inferSchema", "true")
    .option("header", "true")
    .load("C://Users/MZC01-MINJIWOO/Downloads/customer_dataset/marketing_campaign.csv")
            

import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
df: org.apache.spark.sql.DataFrame = [ID: int, Year_Birth: int ... 27 more fields]


In [3]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Year_Birth: integer (nullable = true)
 |-- Education: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Kidhome: integer (nullable = true)
 |-- Teenhome: integer (nullable = true)
 |-- Dt_Customer: string (nullable = true)
 |-- Recency: integer (nullable = true)
 |-- MntWines: integer (nullable = true)
 |-- MntFruits: integer (nullable = true)
 |-- MntMeatProducts: integer (nullable = true)
 |-- MntFishProducts: integer (nullable = true)
 |-- MntSweetProducts: integer (nullable = true)
 |-- MntGoldProds: integer (nullable = true)
 |-- NumDealsPurchases: integer (nullable = true)
 |-- NumWebPurchases: integer (nullable = true)
 |-- NumCatalogPurchases: integer (nullable = true)
 |-- NumStorePurchases: integer (nullable = true)
 |-- NumWebVisitsMonth: integer (nullable = true)
 |-- AcceptedCmp3: integer (nullable = true)
 |-- AcceptedCmp4: integer (nullable = true)
 |-- AcceptedC

In [4]:
def camelToUnderscores(name: String) = "[A-Z\\d]".r.replaceAllIn(name, {m =>
    "_" + m.group(0).toLowerCase()
})

def underscoresToCamel(name: String) = {
  val splitted = name.split("_")
  val converted = splitted.head.toLowerCase() +: splitted.tail.map(_.capitalize)
  converted.mkString
}

val columnsLower = df.columns.map(x => col(x).as(x.toLowerCase))
val columnsCamel = df.columns.map(x => col(x).as(underscoresToCamel(x)))
val dfLower = df.select(columnsLower: _*)
val dfCamel = df.select(columnsCamel: _*)

camelToUnderscores: (name: String)String
underscoresToCamel: (name: String)String
columnsLower: Array[org.apache.spark.sql.Column] = Array(ID AS id, Year_Birth AS year_birth, Education AS education, Marital_Status AS marital_status, Income AS income, Kidhome AS kidhome, Teenhome AS teenhome, Dt_Customer AS dt_customer, Recency AS recency, MntWines AS mntwines, MntFruits AS mntfruits, MntMeatProducts AS mntmeatproducts, MntFishProducts AS mntfishproducts, MntSweetProducts AS mntsweetproducts, MntGoldProds AS mntgoldprods, NumDealsPurchases AS numdealspurchases, NumWebPurchases AS numwebpurchases, NumCatalogPurchases AS numcatalogpurchases, NumStorePurchases AS numstorepurchases, NumWebVisitsMonth AS numwebvisitsmonth, AcceptedCmp3 AS acceptedcmp3, AcceptedCmp4 AS acceptedcmp4, Accepted...


- _* 연산자 : splats 라고 하며 배열, 리스트 등 collection 을 풀어서 가변 인자로 하나씩 넘겨준다.  

In [5]:
dfLower.select("id", "year_birth", "education", "income", "kidhome", "teenhome", "dt_customer", "recency").show(5)

+----+----------+----------+------+-------+--------+-----------+-------+
|  id|year_birth| education|income|kidhome|teenhome|dt_customer|recency|
+----+----------+----------+------+-------+--------+-----------+-------+
|5524|      1957|Graduation| 58138|      0|       0| 04-09-2012|     58|
|2174|      1954|Graduation| 46344|      1|       1| 08-03-2014|     38|
|4141|      1965|Graduation| 71613|      0|       0| 21-08-2013|     26|
|6182|      1984|Graduation| 26646|      1|       0| 10-02-2014|     26|
|5324|      1981|       PhD| 58293|      1|       0| 19-01-2014|     94|
+----+----------+----------+------+-------+--------+-----------+-------+
only showing top 5 rows



In [6]:
dfCamel.select("id", "yearBirth", "education", "income", "kidhome", "teenhome", "dtCustomer").show(5)

+----+---------+----------+------+-------+--------+----------+
|  id|yearBirth| education|income|kidhome|teenhome|dtCustomer|
+----+---------+----------+------+-------+--------+----------+
|5524|     1957|Graduation| 58138|      0|       0|04-09-2012|
|2174|     1954|Graduation| 46344|      1|       1|08-03-2014|
|4141|     1965|Graduation| 71613|      0|       0|21-08-2013|
|6182|     1984|Graduation| 26646|      1|       0|10-02-2014|
|5324|     1981|       PhD| 58293|      1|       0|19-01-2014|
+----+---------+----------+------+-------+--------+----------+
only showing top 5 rows



In [7]:
val dfSelected = dfCamel.select("id", "yearBirth", "education", "income", "kidhome", "teenhome", "dtCustomer", "recency").cache()

dfSelected: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, yearBirth: int ... 6 more fields]


- cache() is an Apache Spark transformation that can be used on a DataFrame, Dataset, or RDD when you want to perform more than one action. cache() caches the specified DataFrame, Dataset, or RDD in the memory of your cluster’s workers. 

In [8]:
case class MarketingUser(userId: Int, 
                         yearBirth: Int, 
                         education: String, 
                         income: Int, 
                         kidhome: Int, 
                         teenhome: Int, 
                         dtCustomer: String,
                         recency: Int)

defined class MarketingUser


- Case Class 는 불변성을 가지고 있어 데이터를 변경하면 새로운 인스턴스가 생성된다. 

In [9]:
// DataFrame[Row] -> Dataset[T]
val dsMarketingUser = dfSelected
  .withColumnRenamed("id", "userId")
  .as[MarketingUser]

dsMarketingUser: org.apache.spark.sql.Dataset[MarketingUser] = [userId: int, yearBirth: int ... 6 more fields]


In [10]:
dfSelected

res4: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, yearBirth: int ... 6 more fields]


In [11]:
dsMarketingUser

res5: org.apache.spark.sql.Dataset[MarketingUser] = [userId: int, yearBirth: int ... 6 more fields]


In [13]:
dsMarketingUser.groupBy(col("education")).agg(sum("income")).show()

+----------+-----------+
| education|sum(income)|
+----------+-----------+
|  2n Cycle|    9526638|
|       PhD|   27005896|
|    Master|   19314900|
|Graduation|   58835937|
|     Basic|    1096538|
+----------+-----------+



In [14]:
val dsUserFiltered = dsMarketingUser.filter(x => x.education == "Master")

dsUserFiltered: org.apache.spark.sql.Dataset[MarketingUser] = [userId: int, yearBirth: int ... 6 more fields]


In [15]:
dsUserFiltered.show(5)

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 8) (MZC01-MINJIWOO executor driver): java.lang.ClassCastException: class $iw cannot be cast to class $iw ($iw is in unnamed module of loader org.apache.spark.repl.ExecutorClassLoader @116ea866; $iw is in unnamed module of loader scala.tools.nsc.interpreter.IMain$TranslatingClassLoader @5d293738)

- scala에서는 NULL 값을 허용하기 위해서 Int 가 아니라 Option[Int] 를 사용해야 하기 때문이다. 

In [16]:
case class MarketingUserRefined(userId: Int, 
                                yearBirth: Int, 
                                education: String, 
                                income: Option[Int], 
                                kidhome: Option[Int], 
                                teenhome: Option[Int], 
                                dtCustomer: String,
                                recency: Option[Int])

defined class MarketingUserRefined


In [17]:

val dsUserRefined = dfSelected
  .withColumnRenamed("id", "userId")
  .as[MarketingUserRefined]


dsUserRefined: org.apache.spark.sql.Dataset[MarketingUserRefined] = [userId: int, yearBirth: int ... 6 more fields]


In [18]:
  
val dsUserFiltered = dsUserRefined.filter(x => x.education == "Master")

dsUserFiltered: org.apache.spark.sql.Dataset[MarketingUserRefined] = [userId: int, yearBirth: int ... 6 more fields]


In [20]:
val dsUserUpdated = dsUserFiltered.map(x => x.copy(yearBirth = x.yearBirth + 3))

dsUserUpdated: org.apache.spark.sql.Dataset[MarketingUserRefined] = [userId: int, yearBirth: int ... 6 more fields]


In [21]:
dsUserFiltered.show()

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 9) (MZC01-MINJIWOO executor driver): java.lang.ClassCastException: class $iw cannot be cast to class $iw ($iw is in unnamed module of loader org.apache.spark.repl.ExecutorClassLoader @116ea866; $iw is in unnamed module of loader scala.tools.nsc.interpreter.IMain$TranslatingClassLoader @5d293738)